In [1]:
import os
import pandas as pd
from ete3 import NCBITaxa
import biothings_client as bt
import tarfile
import csv
import gzip

In [2]:
os.getcwd()

'/Users/bailinzhang/Documents/Wu_Lab/Projects/GMMAD2'

In [3]:
micro_meta_df = pd.read_csv(os.path.join("downloads", "micro_metabolic.csv"), low_memory=False)
meta_gene_df = pd.read_csv(os.path.join("downloads", "meta_gene_net.csv"), low_memory=False)

## Microbe-Disease

In [4]:
with open(os.path.join("downloads", "disease_species.csv"), "r") as f:
    for i, line in enumerate(f, start=1):
        if i == 4:
            print(len(line.strip().split(",")))
            print(line)
            break

29
3,D009765,Obesity,Azorhizobium caulinodans,species,7,0,0,0,0,D006262,Health,3,0.029020807,0.0248571,0.031027083,-0.0248571,Decrease,A status with BODY WEIGHT that is grossly above the acceptable or desirable weight, usually due to accumulation of excess FATS in the body. The standards may vary with age, sex, genetic or cultural background. In the BODY MASS INDEX, a BMI greater than 30.0 kg/m2 is considered obese, and a BMI greater than 40.0 kg/m2 is considered morbidly obese (MORBID OBESITY).,Pseudomonadota,Alphaproteobacteria,Hyphomicrobiales,Xanthobacteraceae,Azorhizobium



In [5]:
def line_generator(in_file):
    """generates lines from a CSV file, yielding each line as a list of strings
    This function opens the specified CSV file, skips the header row, and yields each subsequent line as a list of strings.

    :param in_file: The path to the CSV file.
    :return: An iterator that yields each line of the CSV file as a list of strings.
    """
    with open(in_file, "r") as in_f:
        reader = csv.reader(in_f)
        next(reader)
        for line in reader:
            yield line

In [6]:
micro_disease_path = os.path.join("downloads", "disease_species.csv")
micro_disease = line_generator(micro_disease_path)
for i, line in enumerate(micro_disease):
    if i == 3:
        print(len(line))
        print(line)
    elif i == 15:
        print(len(line))
        print(line)
    elif i == 16:
        print(len(line))
        print(line)
        break

26
['4', 'D008103', 'Liver Cirrhosis', 'Azorhizobium caulinodans', 'species', '7', '0', '0', '0', '0', 'D006262', 'Health', '3', '0.029020807', '0.0248571', '0.031027083', '-0.0248571', 'Decrease', 'Liver disease in which the normal microcirculation', ' the gross vascular anatomy', ' and the hepatic architecture have been variably destroyed and altered with fibrous septa surrounding regenerated or regenerating parenchymal nodules.', 'Pseudomonadota', 'Alphaproteobacteria', 'Hyphomicrobiales', 'Xanthobacteraceae', 'Azorhizobium']
30
['16', 'D001172', 'Arthritis', ' Rheumatoid', 'Azorhizobium caulinodans', 'species', '7', '0', '0', '0', '0', 'D006262', 'Health', '3', '0.029020807', '0.0248571', '0.031027083', '-0.0248571', 'Decrease', 'A chronic systemic disease', ' primarily of the joints', ' marked by inflammatory changes in the synovial membranes and articular structures', ' widespread fibrinoid degeneration of the collagen fibers in mesenchymal tissues', ' and by atrophy and rarefact

In [7]:
def line_generator(in_file):
    """Yield each CSV line as a list of exactly 24 fields,
    rejoining commas inside the 'disease' and 'disease_info' columns.
    """
    EXPECTED_COUNT = 24
    HEAD_COUNT = 2
    TAIL_COUNT = 5
    FIXED_COUNT = 15

    with open(in_file, "r", encoding="utf-8") as f:
        header = next(f).rstrip("\n").split(",")
        print(f"Header column names:{header}")
        next(f)
        for line in f:
            parts = [part.strip() for part in line.split(",")]
            if len(parts) == EXPECTED_COUNT:
                yield parts
            else:
                alteration_idx = next(i for i, p in enumerate(parts) if p in ("Increase", "Decrease"))
                head = parts[:HEAD_COUNT]
                fixed_start = alteration_idx - (FIXED_COUNT - 1)
                disease = ",".join(parts[2:fixed_start])
                fixed = parts[fixed_start: alteration_idx + 1]
                disease_info = ",".join(parts[alteration_idx + 1: len(parts) - TAIL_COUNT])
                tail = parts[-TAIL_COUNT:]
                new_line = head + [disease] + fixed + [disease_info] + tail
                assert len(new_line) == EXPECTED_COUNT, f"Expected {EXPECTED_COUNT} cols, got {len(new_line)}"
                yield new_line


In [8]:
micro_disease = line_generator(micro_disease_path)
for i, line in enumerate(micro_disease):
    if i == 3:
        print(len(line))
        print(line)
    elif i == 15:
        print(len(line))
        print(line)
    elif i == 16:
        print(len(line))
        print(line)
        break

Header column names:['id', 'disease_id', 'disease', 'organism', 'level', 'species_id', 'disease_samples', 'disease_mean', 'disease_median', 'disease_sd', 'health_id', 'health', 'health_samples', 'health_mean', 'health_median', 'health_sd', 'change', 'alteration', 'disease_info', 'phylum', 'class', 'order', 'family', 'genus']
24
['5', 'D001327', 'Autoimmune Diseases', 'Azorhizobium caulinodans', 'species', '7', '0', '0', '0', '0', 'D006262', 'Health', '3', '0.029020807', '0.0248571', '0.031027083', '-0.0248571', 'Decrease', 'Disorders that are characterized by the production of antibodies that react with host tissues or immune effector cells that are autoreactive to endogenous peptides.', 'Pseudomonadota', 'Alphaproteobacteria', 'Hyphomicrobiales', 'Xanthobacteraceae', 'Azorhizobium']
24
['17', 'D043183', 'Irritable Bowel Syndrome', 'Azorhizobium caulinodans', 'species', '7', '0', '0', '0', '0', 'D006262', 'Health', '3', '0.029020807', '0.0248571', '0.031027083', '-0.0248571', 'Decrease

In [9]:
taxids = [line[5] for line in line_generator(micro_disease_path)]
print(len(set(taxids)))

Header column names:['id', 'disease_id', 'disease', 'organism', 'level', 'species_id', 'disease_samples', 'disease_mean', 'disease_median', 'disease_sd', 'health_id', 'health', 'health_samples', 'health_mean', 'health_median', 'health_sd', 'change', 'alteration', 'disease_info', 'phylum', 'class', 'order', 'family', 'genus']
6966


In [10]:
def get_taxon_info(f_path) -> list:
    """retrieves taxonomic information for a given list of taxon IDs from disease_species.csv

    This function reads taxon IDs, removes duplicates, and queries taxonomic info from biothings_client
    to retrieve detailed taxonomic information including scientific name, parent taxid, lineage, and rank.

    :param f_path: Path to disease_species.csv containing the taxids.
    :return: A list of dictionaries containing taxonomic information.
    """
    taxids = [line[5] for line in line_generator(f_path)]
    taxids = set(taxids)
    t = bt.get_client("taxon")
    taxon_info = t.gettaxa(taxids, fields=["scientific_name", "parent_taxid", "lineage", "rank"])
    return taxon_info

In [11]:
notfound = [
    taxon["query"]
    for taxon in get_taxon_info(micro_disease_path)
    if "notfound" in taxon.keys()
]

Header column names:['id', 'disease_id', 'disease', 'organism', 'level', 'species_id', 'disease_samples', 'disease_mean', 'disease_median', 'disease_sd', 'health_id', 'health', 'health_samples', 'health_mean', 'health_median', 'health_sd', 'change', 'alteration', 'disease_info', 'phylum', 'class', 'order', 'family', 'genus']
95
95


In [12]:
print(len(notfound))
print(len(set(notfound)))

95

In [13]:
ncbi = NCBITaxa()
ncbi.update_taxonomy_database()

Updating taxdump.tar.gz from NCBI FTP site (via HTTP)...
Done. Parsing...


Loading node names...
2684591 names loaded.
418729 synonyms loaded.
Loading nodes...
2684591 nodes loaded.
Linking nodes...
Tree is loaded.
Updating database: /Users/bailinzhang/.etetoolkit/taxa.sqlite ...
 2684000 generating entries... 
Uploading to /Users/bailinzhang/.etetoolkit/taxa.sqlite


Inserting synonyms:      45000 

Inserting taxids:       15000        

Inserting taxids:       2680000     

In [14]:
def load_merged_from_tar(tar_gz_path, f_name="merged.dmp"):
    """Parse 'merged.dmp' of taxdump.tar.gz downloaded by ete3.
    Returns a dict {old_taxid: new_taxid}.
    """
    if not os.path.exists(tar_gz_path):
        ncbi = NCBITaxa()
        ncbi.update_taxonomy_database()

    taxid_mapping = {}
    with tarfile.open(tar_gz_path, 'r:gz') as tar:
        f = tar.getmember(f_name)
        with tar.extractfile(f) as fp:
            for line in fp:
                parts = line.decode('utf-8').split('\t')
                old, new = parts[0], parts[2]
                taxid_mapping[old] = new
    return taxid_mapping

In [15]:
def get_current_taxid(old_taxids, merged_mapping):
    taxid_mapping = {}
    for old_taxid in old_taxids:
        taxid_mapping[old_taxid] = merged_mapping[old_taxid]
    return taxid_mapping

In [16]:
mapping = load_merged_from_tar("taxdump.tar.gz")
mapped_taxid = get_current_taxid(notfound, mapping)

In [17]:
if "194866" in mapping:
    print("194866 is in the mapping")
    print(f"current taxid for 194866: {mapping['194866']}")
else:
    print("194866 is not in the mapping")

194866 is in the mapping
current taxid for 194866: 46624


In [18]:
len(mapped_taxid)

95

In [19]:
new_taxids = [new for old, new in mapped_taxid.items()]
print(len(new_taxids))
print(len(set(new_taxids)))

95
86


In [29]:
still_notfound = [taxid for taxid in notfound if taxid not in mapped_taxid.keys()]
print(len(still_notfound))

0


In [20]:
def get_taxon_info(taxids) -> list:
    """retrieves taxonomic information for a given list of taxon IDs from disease_species.csv

    This function reads taxon IDs, removes duplicates, and queries taxonomic info from biothings_client
    to retrieve detailed taxonomic information including scientific name, parent taxid, lineage, and rank.

    :param f_path: Path to disease_species.csv containing the taxids.
    :return: A list of dictionaries containing taxonomic information.
    """
    taxids = set(taxids)
    t = bt.get_client("taxon")
    taxon_info = t.gettaxa(taxids, fields=["scientific_name", "parent_taxid", "lineage", "rank"])
    return taxon_info

In [21]:
new_taxid_mapped = get_taxon_info(new_taxids)
new_taxid_mapped

[{'query': '358',
  '_id': '358',
  '_version': 1,
  'lineage': [358,
   1183400,
   357,
   227290,
   82115,
   356,
   28211,
   1224,
   3379134,
   2,
   131567,
   1],
  'parent_taxid': 1183400,
  'rank': 'species',
  'scientific_name': 'agrobacterium tumefaciens'},
 {'query': '817',
  '_id': '817',
  '_version': 1,
  'lineage': [817,
   816,
   815,
   171549,
   200643,
   976,
   68336,
   1783270,
   3379134,
   2,
   131567,
   1],
  'parent_taxid': 816,
  'rank': 'species',
  'scientific_name': 'bacteroides fragilis'},
 {'query': '3070818',
  '_id': '3070818',
  '_version': 1,
  'lineage': [3070818,
   285107,
   31989,
   204455,
   28211,
   1224,
   3379134,
   2,
   131567,
   1],
  'parent_taxid': 285107,
  'rank': 'species',
  'scientific_name': 'thioclava kandeliae'},
 {'query': '1961',
  '_id': '1961',
  '_version': 1,
  'lineage': [1961, 1883, 2062, 85011, 1760, 201174, 1783272, 2, 131567, 1],
  'parent_taxid': 1883,
  'rank': 'species',
  'scientific_name': 'strep

In [22]:
_ids = [taxon["query"] for taxon in new_taxid_mapped]
still_notfound = [taxid for taxid in new_taxids if taxid not in _ids]

In [23]:
len(set(_ids))

86

In [24]:
taxids = sorted([line[5] for line in line_generator(micro_disease_path)])
print(len(taxids))
print(len(set(taxids)))

Header column names:['id', 'disease_id', 'disease', 'organism', 'level', 'species_id', 'disease_samples', 'disease_mean', 'disease_median', 'disease_sd', 'health_id', 'health', 'health_samples', 'health_mean', 'health_median', 'health_sd', 'change', 'alteration', 'disease_info', 'phylum', 'class', 'order', 'family', 'genus']
508140
6966


## Microbe-Metabolite

In [90]:
def line_generator(in_file, delimiter=","):
    """generates lines from a CSV file, yielding each line as a list of strings
    This function opens the specified CSV file, skips the header row, and yields each subsequent line as a list of strings.

    :param in_file: The path to the CSV file.
    :return: An iterator that yields each line of the CSV file as a list of strings.
    """
    with open(in_file, "r") as in_f:
        reader = csv.reader(in_f, delimiter=delimiter)
        next(reader)
        for line in reader:
            yield line

In [35]:
micro_meta_col_map = dict(enumerate(micro_meta_df.columns))
micro_meta_col_map

{0: 'id',
 1: 'g_micro',
 2: 'organism',
 3: 'g_meta',
 4: 'metabolic',
 5: 'pubchem_compound',
 6: 'pubchem_id',
 7: 'formula',
 8: 'kegg_id',
 9: 'tax_id',
 10: 'phylum',
 11: 'class',
 12: 'order',
 13: 'family',
 14: 'genus',
 15: 'species',
 16: 'species_id',
 17: 'source',
 18: 'smiles_sequence',
 19: 'HMDBID',
 20: 'Origin'}

In [119]:
micro_meta = line_generator(os.path.join("downloads", "micro_metabolic.csv"))
for i, line in enumerate(micro_meta):
    if i == 4:
        print(len(line))
        print(line)
    elif i == 500:
        print(len(line))
        print(line)
    elif i == 1480:
        print(len(line))
        print(line)
        break

21
['5', 'micro1', 'Abiotrophia defectiva ATCC 49176', 'meta21', 'Meso-2,6-Diaminoheptanedioate', '(2R,6S)-2,6-diaminoheptanedioic acid', '99290', 'C7H14N2O4', 'C00680', '592010', 'Bacillota', 'Bacilli', 'Lactobacillales', 'Aerococcaceae', 'Abiotrophia', 'Abiotrophia defectiva', '46125', 'vmh', 'C(CC(C(=O)O)N)CC(C(=O)O)N', 'not available', 'Microbiota']
21
['501', 'micro1', 'Abiotrophia defectiva ATCC 49176', 'meta1286', 'Fructose 1,6-bisphosphate', 'Fructose 1,6-bisphosphate', '5460765', 'C6H10O12P2-4', 'C00354', '592010', 'Bacillota', 'Bacilli', 'Lactobacillales', 'Aerococcaceae', 'Abiotrophia', 'Abiotrophia defectiva', '46125', 'vmh', 'C(C1C(C(C(O1)(COP(=O)([O-])[O-])O)O)O)OP(=O)([O-])[O-]', 'HMDB0001058', 'Host; Microbiota; Food related; Drug related']
21
['1481', 'micro2', 'Achromobacter xylosoxidans A8', 'meta349', '2-octadec-11-enoyl-sn-glycerol 3-phosphate', '2-Octadec-11-Enoyl-Sn-Glycerol 3-Phosphate', 'not available', 'C21H40O7P1', 'not available', '762376', 'Pseudomonadota',

In [64]:
for i, line in enumerate(micro_meta):
    if len(line) != 21:
        print(f"Line {i} has {len(line)} columns: {line}")

In [66]:
no_chem_id = []
for line in micro_meta:
    if "not available" in line[6] and "not available" in line[19]:
        no_chem_id.append(line[4])
print(len(no_chem_id))

303514


In [67]:
print(len(set(no_chem_id)))

1349


In [120]:
pubchem_cids = [line[6] for line in micro_meta if "not available" not in line[6]]
print(len(set(pubchem_cids)))

1571


In [117]:
hmdb_ids = [line[19] for line in micro_meta if "not available" not in line[19] and "not available" in line[6]]
print(len(set(hmdb_ids)))

25


In [73]:
total_metabolites = [line[4] for line in micro_meta]
print(len(set(total_metabolites)))

3200


In [97]:
def get_bigg_metabolite_mapping(in_f):
    bigg_map = {
        line[2].lower():  line[1]
        for line in line_generator(in_f, delimiter="\t")
    }
    return bigg_map

In [98]:
bigg_mapped = get_bigg_metabolite_mapping(os.path.join("downloads", "bigg_models_metabolites.txt"))

In [101]:
len(bigg_mapped)

8771

In [102]:
bigg_mapped_ids = [name.lower() for name in set(no_chem_id) if name.lower() in bigg_mapped]

In [103]:
len(bigg_mapped_ids)

264

In [104]:
set(no_chem_id)

{'Unk_854.4743p_0n_11.9_rb_biocrustexp',
 'released mucin-type O-glycan No 34',
 'Anteisopentadecanoyllipoteichoic acid (n=24), linked, glucose substituted',
 'mucin-type O-glycan No 114',
 'Unk_692.4242p_0n_4.6_rb_biocrustexp',
 'mucin-type O-glycan No 122',
 'mucin-type O-glycan No 12',
 '14-methyl-trans-hexa-dec-2-enoyl-ACP',
 'Isoheptadecanoyllipoteichoic acid (n=24), linked, D-alanine substituted',
 'mucin-type O-glycan No 13',
 '3-Dehydro-Choloyl-CoA',
 'released mucin-type O-glycan No 88',
 'Unk_717.3086p_0n_27.1_rb_biocrustexp',
 'C6h13n3o2_160.1076p_0n_10_rb_biocrustexp',
 'released mucin-type O-glycan No 33',
 'Chondroitin Sulfate B / Dermatan Sulfate (Idoa2S-GalNac4S) Proteoglycan',
 '5-Hydroxyvalerate,5-Hydroxypentanoate',
 'released mucin-type O-glycan No 25',
 'Fa4coa,Anteiso-C15:0 CoA   Anteisopentadecanoyl-CoA',
 'Cardiolipin (13-methyl-tetradecanoyl, iso-C15)',
 'Unk_123.0545p_0n_7.6_rb_biocrustexp',
 '1,2-Diacyl-sn-glycerol (dioctadec-11-enoyl, n-C18:1)',
 'mucin-type